In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CALC BENIGN MALIGN NORMAL CLASSIFIER USING MAMMOGRAM PATCHES IN CBIS-DDSM
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v008'

In [2]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications.nasnet import NASNetMobile
base_model=NASNetMobile(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=ReLU()(x)
preds=Dense(5,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [4]:
len(model.layers)

776

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=6

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/train', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/valid', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)


Found 2170 images belonging to 5 classes.
Found 545 images belonging to 5 classes.


In [7]:
#Callbacks
#checkpoint_path="resnet50_train4/cp.ckpt"
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mas_cal_ben_mal_class_nasnet50_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
361/361 [==============================] - 169s 318ms/step - loss: 1.4427 - accuracy: 0.3831 - val_loss: 1.2046 - val_accuracy: 0.5019


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/3
361/361 [==============================] - 59s 162ms/step - loss: 1.2997 - accuracy: 0.4293 - val_loss: 1.2021 - val_accuracy: 0.4796
Epoch 3/3
361/361 [==============================] - 52s 143ms/step - loss: 1.2506 - accuracy: 0.4644 - val_loss: 1.1323 - val_accuracy: 0.4907


In [9]:
#Training stage 2
for layer in model.layers[:520]:
    layer.trainable=False
for layer in model.layers[520:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/10
361/361 [==============================] - 120s 235ms/step - loss: 1.1785 - accuracy: 0.4880 - val_loss: 1.0834 - val_accuracy: 0.5370
Epoch 2/10
361/361 [==============================] - 85s 236ms/step - loss: 0.9393 - accuracy: 0.5920 - val_loss: 1.2522 - val_accuracy: 0.5093
Epoch 3/10
361/361 [==============================] - 87s 241ms/step - loss: 0.8741 - accuracy: 0.6354 - val_loss: 1.3979 - val_accuracy: 0.4722
Epoch 4/10
361/361 [==============================] - 86s 238ms/step - loss: 0.8053 - accuracy: 0.6525 - val_loss: 1.0297 - val_accuracy: 0.6204
Epoch 5/10
361/361 [==============================] - 87s 240ms/step - loss: 0.7820 - accuracy: 0.6673 - val_loss: 0.9428 - val_accuracy: 0.5852
Epoch 6/10
361/361 [==============================] - 87s 240ms/step - loss: 0.7429 - accuracy: 0.6811 - val_loss: 1.3347 - val_accuracy: 0.4778
Epoch 7/10
361/361 [==============================] - 87s 240ms/step - loss: 0.7078 - accuracy: 0.7006 - val_loss: 0.9866 - val_a

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/150
361/361 [==============================] - 198s 431ms/step - loss: 0.9705 - accuracy: 0.6077 - val_loss: 0.8076 - val_accuracy: 0.6796
Epoch 2/150
361/361 [==============================] - 149s 411ms/step - loss: 0.8079 - accuracy: 0.6599 - val_loss: 0.9185 - val_accuracy: 0.6296
Epoch 3/150
361/361 [==============================] - 150s 415ms/step - loss: 0.7555 - accuracy: 0.6821 - val_loss: 1.0193 - val_accuracy: 0.5852
Epoch 4/150
361/361 [==============================] - 148s 411ms/step - loss: 0.7472 - accuracy: 0.6913 - val_loss: 1.1918 - val_accuracy: 0.5148
Epoch 5/150
361/361 [==============================] - 137s 380ms/step - loss: 0.7056 - accuracy: 0.7079 - val_loss: 1.3779 - val_accuracy: 0.3852
Epoch 6/150
361/361 [==============================] - 137s 379ms/step - loss: 0.7037 - accuracy: 0.7019 - val_loss: 1.4491 - val_accuracy: 0.3444
Epoch 7/150
361/361 [==============================] - 149s 411ms/step - loss: 0.6766 - accuracy: 0.7167 - val_loss: 1

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/valid', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred = np.argmax(Y_pred, axis=1)
ras=roc_auc_score(valid_generator.classes, Y_pred,multi_class='ovr')


print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('ROC AUC score:', ras)



Found 545 images belonging to 5 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[ 58  24   5  15   7]
 [  4  65  10  28   2]
 [  1   3  41  56   8]
 [  1   1   8  99   0]
 [  0   0   0   1 108]]
Accuracy: 0.6807339449541284
ROC AUC score: 0.9182770810537834


In [13]:
train_generator.filenames

['calc_ben\\calc_ben_0001.png',
 'calc_ben\\calc_ben_0002.png',
 'calc_ben\\calc_ben_0003.png',
 'calc_ben\\calc_ben_0004.png',
 'calc_ben\\calc_ben_0006.png',
 'calc_ben\\calc_ben_0007.png',
 'calc_ben\\calc_ben_0008.png',
 'calc_ben\\calc_ben_0009.png',
 'calc_ben\\calc_ben_0017.png',
 'calc_ben\\calc_ben_0019.png',
 'calc_ben\\calc_ben_0020.png',
 'calc_ben\\calc_ben_0022.png',
 'calc_ben\\calc_ben_0023.png',
 'calc_ben\\calc_ben_0024.png',
 'calc_ben\\calc_ben_0025.png',
 'calc_ben\\calc_ben_0034.png',
 'calc_ben\\calc_ben_0035.png',
 'calc_ben\\calc_ben_0036.png',
 'calc_ben\\calc_ben_0038.png',
 'calc_ben\\calc_ben_0041.png',
 'calc_ben\\calc_ben_0049.png',
 'calc_ben\\calc_ben_0050.png',
 'calc_ben\\calc_ben_0051.png',
 'calc_ben\\calc_ben_0053.png',
 'calc_ben\\calc_ben_0054.png',
 'calc_ben\\calc_ben_0056.png',
 'calc_ben\\calc_ben_0057.png',
 'calc_ben\\calc_ben_0065.png',
 'calc_ben\\calc_ben_0067.png',
 'calc_ben\\calc_ben_0069.png',
 'calc_ben\\calc_ben_0071.png',
 'calc_b

In [14]:
valid_generator.filenames

['calc_ben\\calc_ben_0005.png',
 'calc_ben\\calc_ben_0018.png',
 'calc_ben\\calc_ben_0021.png',
 'calc_ben\\calc_ben_0033.png',
 'calc_ben\\calc_ben_0037.png',
 'calc_ben\\calc_ben_0039.png',
 'calc_ben\\calc_ben_0040.png',
 'calc_ben\\calc_ben_0052.png',
 'calc_ben\\calc_ben_0055.png',
 'calc_ben\\calc_ben_0066.png',
 'calc_ben\\calc_ben_0068.png',
 'calc_ben\\calc_ben_0070.png',
 'calc_ben\\calc_ben_0072.png',
 'calc_ben\\calc_ben_0081.png',
 'calc_ben\\calc_ben_0082.png',
 'calc_ben\\calc_ben_0089.png',
 'calc_ben\\calc_ben_0098.png',
 'calc_ben\\calc_ben_0102.png',
 'calc_ben\\calc_ben_0114.png',
 'calc_ben\\calc_ben_0115.png',
 'calc_ben\\calc_ben_0129.png',
 'calc_ben\\calc_ben_0137.png',
 'calc_ben\\calc_ben_0146.png',
 'calc_ben\\calc_ben_0147.png',
 'calc_ben\\calc_ben_0149.png',
 'calc_ben\\calc_ben_0150.png',
 'calc_ben\\calc_ben_0152.png',
 'calc_ben\\calc_ben_0166.png',
 'calc_ben\\calc_ben_0167.png',
 'calc_ben\\calc_ben_0184.png',
 'calc_ben\\calc_ben_0193.png',
 'calc_b

In [15]:
#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

[[0.8184229  0.1099148  0.04239077 0.02557948 0.00369213]
 [0.84683126 0.1290672  0.01049515 0.01006343 0.0035429 ]
 [0.07847639 0.17461373 0.33507404 0.4031986  0.00863731]
 ...
 [0.00445801 0.00411307 0.02830527 0.0137256  0.9493981 ]
 [0.00333352 0.0039846  0.10230965 0.05148235 0.8388899 ]
 [0.00386752 0.00374385 0.00767132 0.00221013 0.98250717]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2